In [1]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

In [3]:
data_raw = pd.read_csv("./Movie_classification.csv")
print(data_raw.shape)

(506, 19)


In [8]:
data_no_missing = data_raw.dropna()
print(data_no_missing.shape)

(494, 19)


In [10]:
data = data_no_missing.drop(["Genre", "3D_available"], axis=1)
print(data.shape)

(494, 17)


In [11]:
data.head()

,Marketing expense,Production expense,Multiplex coverage,Budget,Movie_length,Lead_ Actor_Rating,Lead_Actress_rating,Director_rating,Producer_rating,Critic_rating,Trailer_views,Time_taken,Twitter_hastags,Avg_age_actors,Num_multiplex,Collection,Start_Tech_Oscar
0,20.1264,59.62,0.462,36524.125,138.7,7.825,8.095,7.910,7.995,7.94,527367,109.60,223.840,23,494,48000,1
1,20.5462,69.14,0.531,35668.655,152.4,7.505,7.650,7.440,7.470,7.44,494055,146.64,243.456,42,462,43200,0
2,20.5458,69.14,0.531,39912.675,134.6,7.485,7.570,7.495,7.515,7.44,547051,147.88,2022.400,38,458,69400,1
3,20.6474,59.36,0.542,38873.890,119.3,6.895,7.035,6.920,7.020,8.26,516279,185.36,225.344,45,472,66800,1
4,21.3810,59.36,0.542,39701.585,127.7,6.920,7.070,6.815,7.070,8.26,531448,176.48,225.792,55,395,72400,1


In [13]:
X=data.drop("Start_Tech_Oscar", axis=1)
print(X.shape)

y=data.Start_Tech_Oscar
print(y.shape)

(494, 16)
(494,)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=127)

In [17]:
scaler = StandardScaler()
X_train_scl = scaler.fit_transform(X_train)
X_test_scl = scaler.transform(X_test)

In [19]:
model_1 = GaussianNB()
fit_1 =model_1.fit(X_train_scl,y_train)

In [23]:
predict_1= fit_1.predict_proba(X_test_scl)
predict_1_score =predict_1[:,1]
auc_1 = roc_auc_score(y_test, predict_1_score)
print(auc_1)

0.5850840336134454


In [21]:
model_2 =KNeighborsClassifier()
fit_2 = model_2.fit(X_train_scl, y_train)

In [24]:
predict_2 =fit_2.predict_proba(X_test_scl)
predict_2_score = predict_2[:,1]

auc_2 = roc_auc_score(y_test, predict_2_score)
print(auc_2)

0.6528361344537815


In [25]:
model_3 = tree.DecisionTreeClassifier(criterion='entropy')
fit_3 = model_3.fit(X_train_scl, y_train)
predict_3 = fit_3.predict_proba(X_test_scl)
predict_3_score = predict_3[:,1]

auc_3 = roc_auc_score(y_test, predict_3_score)
print(auc_3)

0.618172268907563


In [26]:
def predict_proba_ensamble(X, model_1, model_2, model_3):
    y_pred_1 = model_1.predict_proba(X)[:,1]
    y_pred_2 = model_2.predict_proba(X)[:,1]
    y_pred_3 = model_3.predict_proba(X)[:,1]
    result=(y_pred_1+y_pred_2+y_pred_3)/3
    return result

In [27]:
predict_ensamble = predict_proba_ensamble(X_test_scl, fit_1, fit_2, fit_3)
auc_ensamnble = roc_auc_score(y_test, predict_ensamble)
print(auc_ensamnble)

0.6859243697478992


In [28]:
threshold = 0.5
predict_ensamble_cat=[1 if (x>= threshold) else 0 for x in predict_ensamble]

accuracy_ensamble = accuracy_score(y_test, predict_ensamble_cat)
print(accuracy_ensamble)

confusion_matrix_ensamble= confusion_matrix(y_test, predict_ensamble_cat)
print(confusion_matrix_ensamble)

0.6129032258064516
[[38 18]
 [30 38]]
